In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fashionmnist/t10k-labels-idx1-ubyte
/kaggle/input/fashionmnist/t10k-images-idx3-ubyte
/kaggle/input/fashionmnist/fashion-mnist_test.csv
/kaggle/input/fashionmnist/fashion-mnist_train.csv
/kaggle/input/fashionmnist/train-labels-idx1-ubyte
/kaggle/input/fashionmnist/train-images-idx3-ubyte


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}")


Using cuda


In [4]:
train_df = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_train.csv')
test_df = pd.read_csv('/kaggle/input/fashionmnist/fashion-mnist_test.csv')

print(train_df.head())
print(test_df.head())

print("Dataset loaded successfully")

   label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0      2       0       0       0       0       0       0       0       0   
1      9       0       0       0       0       0       0       0       0   
2      6       0       0       0       0       0       0       0       5   
3      0       0       0       0       1       2       0       0       0   
4      3       0       0       0       0       0       0       0       0   

   pixel9  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0        30        43         0   
3       0  ...         3         0         0         0         0         1   
4       0  ...         0         0         0         0         0         0   

   pixel781  pixel782  pixel783  pixel784  
0         0         0         

In [5]:
X = train_df.iloc[:, 1:].values
y = train_df.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Scaling the values.
X_train = X_train/255.0
X_test = X_test/255.0



In [6]:
class customDataset(Dataset):

    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]

print('Custom Dataset class created')

Custom Dataset class created


In [7]:
train_dataset_object = customDataset(X_train, y_train)
test_dataset_objects = customDataset(X_test, y_test)

train_loader = DataLoader(train_dataset_object, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset_objects, batch_size=32, shuffle=False)

print("Dataset and Dataloader objects created.")

Dataset and Dataloader objects created.


In [8]:
## NN class
class NeuralANN(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        self.structure = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.structure(x)

print("NN Class created successfully")

NN Class created successfully


In [9]:
learning_rate = 0.1
epochs = 100

In [10]:
ANNModel = NeuralANN(X_train.shape[1])
ANNModel = ANNModel.to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = optim.SGD(ANNModel.parameters(), lr=learning_rate)

print("Model initialization done")

Model initialization done


In [11]:
for epoch in range(epochs):
    total_epoch_loss = 0
    for batch_features, batch_labels in train_loader:

        #Moving the features and labels to cuda
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        
        #Forward pass
        y_pred = ANNModel(batch_features)

        #Loss calculation
        loss = loss_function(y_pred, batch_labels)

        #Backward pass
        optimizer.zero_grad()
        loss.backward()

        #Update grads
        optimizer.step()

        total_epoch_loss += loss.item()
    
    avg_loss = total_epoch_loss/len(train_loader)
    if(epoch%10 == 0):
        print(f"Epoch: {epoch} Loss: {avg_loss}")
print("Model trained successfully\n")

Epoch: 0 Loss: 0.6349930875301361
Epoch: 10 Loss: 0.26839526614298426
Epoch: 20 Loss: 0.20650795155763627
Epoch: 30 Loss: 0.16863190730909505
Epoch: 40 Loss: 0.13701548981464778
Epoch: 50 Loss: 0.11563370788811396
Epoch: 60 Loss: 0.10016697603852177
Epoch: 70 Loss: 0.09069666740274988
Epoch: 80 Loss: 0.0772382070836223
Epoch: 90 Loss: 0.06849827821496486
Model trained successfully



In [12]:
print(ANNModel.eval())
    
total = 0
correct = 0

with torch.no_grad():

    for batch_features, batch_labels in test_loader:
        
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        
        output = ANNModel(batch_features)
        

        _, predicted = torch.max(output, 1) #Extracting the label for the maximum probabilistic value returned by the model.
        
        total = total + batch_labels.shape[0]
        correct = correct + (predicted == batch_labels).sum().item()
    

accuracy = correct/total
print(f"Test Accuracy: {accuracy * 100:.4f}%")

NeuralANN(
  (structure): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)
Test Accuracy: 88.5583%
